In [30]:
hour_table = {
    (0, 0): 0,
    (0, 1): 14,
    (0, 2): 15,
    (0, 3): 17,
    (0, 4): 16,
    
    (1, 0): 14,
    (1, 1): 0,
    (1, 2): 24,
    (1, 3): 8,
    (1, 4): 36,
    
    (2, 0): 15,
    (2, 1): 24,
    (2, 2): 0,
    (2, 3): 34,
    (2, 4): 4,
    
    (3, 0): 17,
    (3, 1): 8,
    (3, 2): 34,
    (3, 3): 0,
    (3, 4): 30,
    
    (4, 0): 16,
    (4, 1): 36,
    (4, 2): 4,
    (4, 3): 30,
    (4, 4): 0
}
    

In [216]:
from typing import List
from more_itertools import pairwise


def get_hour(start, end):
    coord = (start, end)
    return hour_table[coord]


def get_index(country_name):
    mapper = {c_name: i for i, c_name in enumerate(c_remained)}
    return mapper.get(country_name)


def get_hour_total():
    if not route:
        return 0
    else:
        index_pairs = [(get_index(c1), get_index(c2)) for c1, c2 in pairwise(route)]
        return sum([get_hour(idx1, idx2) for idx1, idx2 in index_pairs])
    

def print_result(hour_total: int, route: List[str]):
    route_str = " -> ".join(route)
    route_str += f": {hour_total}"
    print(route_str)

In [217]:
from dataclasses import dataclass

@dataclass
class Country:
    name: str
    index: int
    hour_to_take: int

In [218]:
class Comparable:
    v1: float
    v2: float
    
    def compare(self, v1, v2) -> float:
        if v1 < v2:
            return v1
        else:
            return v2

In [219]:
from collections import deque
from typing import Deque

class PriorityQueue:
    def __init__(self):
        self.c_remained = deque()
        
    def __len__(self):
        return len(self.c_remained)
    
    def pop(self):
        last = self.c_remained.pop()
        return last
    
    def popleft(self):
        first = self.c_remained.popleft()
        return first

    def append(self, c):
        self.c_remained.append(c)
        self.c_remained = sorted(
            self.c_remained, 
            key=lambda country: country.hour_to_take
        )
        self.c_remained = deque(self.c_remained)
        return None

In [222]:
c_remained = ["NP", "IS", "CA", "UK", "US"]
route = list()
hour_best = None

def branchbound():
    global route, hour_best
    
    start_country_str = "NP"
    route.append(start_country_str)
    branchbound_recursion()
    return None
    

def branchbound_recursion():
    global route, hour_best
    
    if len(route) == len(c_remained):
        hour_total = get_hour_total()
        if not hour_best or hour_total < hour_best:
            hour_best = hour_total
            print_result(hour_total, route)
        else:
            print("[X]", end=" ")
            print_result(hour_total, route)
        
        return None
    else:
        hour_total = get_hour_total()
        if hour_best and hour_total >= hour_best:
            print("(bound)", end=" ")
            print_result(hour_total, route)
            return None
        
    pq = PriorityQueue()
    
    start_country_str = route[-1]
    for end_country_str in c_remained:
        if end_country_str in route:
            continue

        index = get_index(end_country_str)
        hour = get_hour(get_index(start_country_str), index)
        pq.append(Country(end_country_str, index, hour))
    
    while True:
        if len(pq) == 0:
            break
        c_next = pq.popleft()
        route.append(c_next.name)
        branchbound_recursion()
        route.remove(c_next.name)

In [223]:
branchbound()

NP -> IS -> UK -> US -> CA: 56
(bound) NP -> IS -> UK -> CA: 56
[X] NP -> IS -> CA -> US -> UK: 72
(bound) NP -> IS -> CA -> UK: 72
[X] NP -> IS -> US -> CA -> UK: 88
(bound) NP -> IS -> US -> UK: 80
[X] NP -> CA -> US -> UK -> IS: 57
[X] NP -> CA -> US -> IS -> UK: 63
[X] NP -> CA -> IS -> UK -> US: 77
(bound) NP -> CA -> IS -> US: 75
(bound) NP -> CA -> UK -> IS: 57
(bound) NP -> CA -> UK -> US: 79
NP -> US -> CA -> IS -> UK: 52
(bound) NP -> US -> CA -> UK: 54
(bound) NP -> US -> UK -> IS: 54
(bound) NP -> US -> UK -> CA: 80
(bound) NP -> US -> IS: 52
[X] NP -> UK -> IS -> CA -> US: 53
(bound) NP -> UK -> IS -> US: 61
[X] NP -> UK -> US -> CA -> IS: 75
(bound) NP -> UK -> US -> IS: 83
(bound) NP -> UK -> CA -> US: 55
(bound) NP -> UK -> CA -> IS: 75
